In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

True

In [3]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

c:\ChatBot using LangGraph\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [5]:
def generate_joke(state: JokeState):

    prompt=f"Generate a joke about {state['topic']}"
    response=model.invoke(prompt).content
    return {'joke': response}

In [6]:
def generate_explanation(state: JokeState):

    prompt=f"Explain the joke: {state['joke']}"
    response=model.invoke(prompt).content
    return {'explanation': response}

In [14]:
graph=StateGraph(JokeState)
checkpointer= InMemorySaver()

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

workflow=graph.compile(checkpointer=checkpointer)



In [17]:
config2= {'configurable': {'thread_id': '2'}}

workflow.invoke({'topic': 'life'}, config=config2)



{'topic': 'life',
 'joke': " Why don't we ever worry about the middle age? Because it's the prime of life! Just remember, the secret of staying young is to live honestly, laugh often, and love deeply... and keep a good doctor on speed dial!",
 'explanation': ' This joke plays on the double meaning of the term "middle age." Typically, "middle age" is considered a period in a person\'s life, usually between the ages of 40 and 60, that is characterized by various physical, emotional, and social changes. However, in the context of numbers, "middle age" refers to the age that is exactly in the middle of a range.\n\nIn this case, the speaker humorously suggests that we should not worry about "middle age" because it is the "prime of life." The term "prime" has two meanings here: it can refer to the best or most important period of a person\'s life, but it can also refer to a number that has only two distinct prime factors (in this case, 2 and itself). By using the term "prime" in this way, th

In [19]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'life', 'joke': " Why don't we ever worry about the middle age? Because it's the prime of life! Just remember, the secret of staying young is to live honestly, laugh often, and love deeply... and keep a good doctor on speed dial!", 'explanation': ' This joke plays on the double meaning of the term "middle age." Typically, "middle age" is considered a period in a person\'s life, usually between the ages of 40 and 60, that is characterized by various physical, emotional, and social changes. However, in the context of numbers, "middle age" refers to the age that is exactly in the middle of a range.\n\nIn this case, the speaker humorously suggests that we should not worry about "middle age" because it is the "prime of life." The term "prime" has two meanings here: it can refer to the best or most important period of a person\'s life, but it can also refer to a number that has only two distinct prime factors (in this case, 2 and itself). By using the term "pri

In [20]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'trader', 'joke': " Why don't traders ever play hide and seek?\n\nBecause good luck hiding when the market always knows where you've put your money!", 'explanation': ' This joke is based on the nature of trading and the stock market. Traders invest money in various assets (stocks, bonds, commodities, etc.) with the hope of making a profit. The market, in this context, refers to the collective behavior of all buyers and sellers in the marketplace.\n\nThe punchline "Because good luck hiding when the market always knows where you\'ve put your money!" is a humorous way of saying that it\'s impossible to keep your trading activities secret because the market (i.e., other traders and investors) can see where you\'ve invested your money through the buying and selling of assets. In other words, your trades are public information, and it\'s challenging to hide your investment decisions from the market.'}, next=(), config={'configurable': {'thread_id': '1', 'check